# Interaction with the file system (2)

## interesting alternatives: the `pathlib` and `shutil` modules

These modules work a bit differently than the `os` module. While the `os` module is purely functional and the path is fed in as a string, the `pathlib` transforms the given path into an _object_ which then offers a number of _methods_ related to that object.

**import pathlib and shutil**

In [ ]:
import pathlib
import shutil

### test if a file or path exists

In [ ]:
file = pathlib.Path('/home/an/unknown/file/somewhere.txt') 

In [ ]:
file.exists()

In [ ]:
directory = pathlib.Path('.')

In [ ]:
directory.exists()

In [ ]:
directory.is_dir()

### Concatenate paths, using `/`

In [ ]:
my_hello_world = directory / "my_modules" / "my_hello_world.py"

In [ ]:
my_hello_world.exists()

the `absolute()` method returns, not surprisingly, the absolute path. Well, not exactly. It returns a `PosixPath` object:

In [ ]:
my_hello_world.absolute()

We can get a normal string representation of it:

In [ ]:
my_hello_world.absolute().as_posix()

... or a URI represention:

In [ ]:
my_hello_world.absolute().as_uri()

### show and change file access flags: `chmod`

In [ ]:
file = pathlib.Path('_stat_info_testfile')
file.touch()

In [ ]:
oct(file.stat().st_mode & 0o777)

In [ ]:
file.chmod(0o600)

In [ ]:
oct(file.stat().st_mode & 0o777)

In [ ]:
file.unlink()

### change ownership of a file: `chown`

In [ ]:
chown_testfile = pathlib.Path('_pathlib_ownership_testfile')
chown_testfile.touch()

In [ ]:
print("owner:", chown_testfile.owner())
print("group:", chown_testfile.group())

In [ ]:
shutil.chown(path=chown_testfile, group='everyone')

In [ ]:
print("owner:", chown_testfile.owner())
print("group:", chown_testfile.group())

In [ ]:
chown_testfile.unlink()

### copy files: `cp`

In [ ]:
import shutil
import os
source = os.listdir(".")
destination = "backup_folder"

if not os.path.exists(destination):
    os.mkdir(destination)
    
for file in source:
    if file.endswith(".ipynb"):
        shutil.copy(file,destination)

In [ ]:
os.listdir(destination)

In [ ]:
shutil.rmtree(destination, ignore_errors=True)

## match file patterns: `glob`

In [ ]:
current_dir = pathlib.Path('.')

In [ ]:
for file in current_dir.glob("0?_*.ipynb"):
    print(file)

### copy a directory recursively

prepare a nested directory...

In [ ]:
import os
source_dir = "start/of/some/deeply/nested/directory"
os.makedirs(source_dir)

create the destination directory

In [ ]:
destination_dir = "destination_directory"
os.mkdir(destination_dir)

Try to execute the cell above again. What error do you get? How can we avoid the error?

**catch the `FileExistsError`**

In [ ]:
import os
try:
    os.mkdir(destination_dir)
except FileExistsError:  # catch this specific error
    pass                 # resolve things. In our case: do nothing

now, we recursively copy the `source_dir` to the `destination_dir`:

In [ ]:
shutil.copytree(source_dir, destination_dir)

**???**

now you realize, in the Python standard library, there are sometimes **very annoying limitations**. The code below will  work without annoyances, but with Python 3.8 and onward only, otherwise it will complain again `TypeError: copytree() got an unexpected keyword argument 'dirs_exist_ok'`

In [ ]:
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

**Conclusion: Google and StackOverflow are your friends.** Don't hesistate to consult them for the (currently) best solution to your problem :)

Of course, there exists a workaround which works nicely and according to the **DWIM** principle: **D**o **W**hat **I** **M**ean

In [ ]:
from distutils.dir_util import copy_tree

copy_tree("start", destination_dir)

In [ ]:
shutil.rmtree(destination_dir, ignore_errors=True)

In [ ]:
shutil.rmtree("start", ignore_errors=True)